In [1]:
from sklearn import preprocessing
from sklearn import metrics

In [2]:
import numpy as np
import pandas as pd
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [3]:
import matplotlib.pyplot as plt
import os, os.path, sys

In [4]:
path = r"podaci\0k\X"

In [5]:
dir = os.listdir(path)

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

Potencijalni time_stepovi su (1, 2, 4, 5, 8, 10, 20, 25, 40, 50, 61, 100), kako bi bili deljivi sa 61000

best step = 20

In [7]:
time_steps = 2

In [8]:
input_shape = (time_steps,1)

In [9]:
def create_timeseries_split(data_matrix, time_steps):

  # Get the number of samples in the data
  num_samples = len(data_matrix) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data_matrix[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data_matrix[i+time_steps, 0]

  return x_train, y_train

In [173]:
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

In [174]:
data_matrix = np.concatenate([pd.read_csv(path + "\\" + file, header=None).astype('int')
                              for file in file_names], axis=0)
data_matrix = scaler.fit_transform(data_matrix)  # Scale entire dataset

# Split data
train_idx = int(0.8 * len(data_matrix))
x_train, y_train = create_timeseries_split(data_matrix[:train_idx], time_steps)
x_test, y_test = create_timeseries_split(data_matrix[train_idx:], time_steps)

In [175]:
#scale the y back cuz
y_train = scaler.inverse_transform(y_train)
y_test = scaler.inverse_transform(y_test)

In [176]:
np.random.seed(1234)

In [226]:
model = Sequential()
model.add(SimpleRNN(units = 16, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
model.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
model.add(Dense(units = 32, activation=keras.layers.LeakyReLU()))
model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer = 'adam')

In [227]:
history = model.fit(x_train, y_train, epochs=10, batch_size = 64, verbose = 2)

Epoch 1/10
24143/24143 - 49s - loss: 192468720.0000 - 49s/epoch - 2ms/step
Epoch 2/10
24143/24143 - 41s - loss: 6206.1167 - 41s/epoch - 2ms/step
Epoch 3/10
24143/24143 - 44s - loss: 6245.6011 - 44s/epoch - 2ms/step
Epoch 4/10
24143/24143 - 43s - loss: 6238.9204 - 43s/epoch - 2ms/step
Epoch 5/10
24143/24143 - 41s - loss: 6241.9419 - 41s/epoch - 2ms/step
Epoch 6/10
24143/24143 - 41s - loss: 6231.5933 - 41s/epoch - 2ms/step
Epoch 7/10
24143/24143 - 42s - loss: 6228.9497 - 42s/epoch - 2ms/step
Epoch 8/10
24143/24143 - 41s - loss: 6223.3467 - 41s/epoch - 2ms/step
Epoch 9/10
24143/24143 - 40s - loss: 6256.7227 - 40s/epoch - 2ms/step
Epoch 10/10
24143/24143 - 41s - loss: 6238.7275 - 41s/epoch - 2ms/step


In [228]:
train_scores = model.evaluate(x_test, y_test)

12072/12072 [==============================] - 16s 1ms/step - loss: 6130.4258


In [229]:
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)

12072/12072 [==============================] - 16s 1ms/step


In [138]:
int(0.8 * len(data_matrix))

1545130

In [108]:
max_koord = pd.read_csv(r"C:\Users\mata2\Desktop\jupyter\Master rad\Matija_Grulovic\podaci\0k\X_max_koord_0k.txt", names=['x osa'])
min_koord = pd.read_csv(r"C:\Users\mata2\Desktop\jupyter\Master rad\Matija_Grulovic\podaci\0k\X_min_koord_0k.txt", names=['x osa'])

In [109]:
max_koord_scaled = scaler.transform(max_koord)
min_koord_scaled = scaler.transform(min_koord)

D:\Users\mata2\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
D:\Users\mata2\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [110]:
#plt.figure(figsize=(15,5))
#plt.plot(min_koord[8300:8500])
#plt.plot(max_koord[8300:8500])

In [189]:
Y_ind = np.arange(time_steps, len(min_koord), time_steps)
Y_ind

array([    2,     4,     6, ..., 60994, 60996, 60998])

In [202]:
train_predict.shape

(1545128, 1)

In [224]:
num_correct = 0
for i in range(len(train_predict)):
    if train_predict[i][0] < y_train[i][0]+5:
        if train_predict[i][0] > y_train[i][0]-5:
            #print(np.round(y_train[i][0],2)-5, np.round(train_predict[i][0],2), np.round(y_train[i][0],2)+5)
            num_correct += 1

In [225]:
num_correct_test = 0
for i in range(len(test_predict)):
    if test_predict[i][0] < y_test[i][0]+5:
        if test_predict[i][0] > y_test[i][0]-5:
            #print(np.round(y_test[i][0],2)-5, np.round(test_predict[i][0],2), np.round(y_test[i][0],2)+5)
            num_correct_test += 1

In [223]:
print(num_correct_test/(len(test_predict)), num_correct_test)

0.15074259412189572 58229


In [216]:
print(num_correct/(len(train_predict)), num_correct)

0.15177383362413988 234510


In [217]:
for i in range(len(y_train)):
    print(np.round(y_train[i][0],2)-5, np.round(train_predict[i][0],2), np.round(y_train[i][0],2)+5)

-5.0 -1.85 5.0
-3.0 -1.85 7.0
0.0 -1.84 10.0
5.0 -1.64 15.0
13.0 -1.34 23.0
23.0 -0.84 33.0
32.0 -0.06 42.0
34.0 3.43 44.0
34.0 11.79 44.0
35.0 14.03 45.0
37.0 14.07 47.0
43.0 15.26 53.0
62.0 17.72 72.0
86.0 25.12 96.0
102.0 47.29 112.0
108.0 74.81 118.0
111.0 92.99 121.0
112.0 99.83 122.0
118.0 103.24 128.0
132.0 104.57 142.0
151.0 111.8 161.0
174.0 128.27 184.0
202.0 149.37 212.0
229.0 173.92 239.0
239.0 203.59 249.0
247.0 231.7 257.0
251.0 242.24 261.0
258.0 250.6 268.0
275.0 254.94 285.0
300.0 262.68 310.0
328.0 280.97 338.0
357.0 307.59 367.0
389.0 337.32 399.0
411.0 368.18 421.0
424.0 400.54 434.0
435.0 422.18 445.0
445.0 435.07 455.0
461.0 446.0 471.0
484.0 456.16 494.0
515.0 472.33 525.0
548.0 495.51 558.0
575.0 526.47 585.0
605.0 559.16 615.0
637.0 585.79 647.0
657.0 613.94 667.0
677.0 645.05 687.0
697.0 664.99 707.0
720.0 684.92 730.0
747.0 704.95 757.0
779.0 728.01 789.0
807.0 755.08 817.0
831.0 786.85 841.0
861.0 814.63 871.0
892.0 838.75 902.0
925.0 868.68 935.0
958.0 899.

9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985.02 10006.0
9996.0 9985

171946.0 171817.1 171956.0
171997.0 171865.2 172007.0
172049.0 171917.16 172059.0
172099.0 171968.19 172109.0
172148.0 172020.12 172158.0
172197.0 172070.08 172207.0
172246.0 172119.06 172256.0
172296.0 172168.08 172306.0
172349.0 172217.1 172359.0
172400.0 172267.19 172410.0
172450.0 172320.1 172460.0
172499.0 172371.08 172509.0
172548.0 172421.05 172558.0
172598.0 172470.05 172608.0
172649.0 172519.06 172659.0
172700.0 172569.1 172710.0
172750.0 172620.08 172760.0
172800.0 172671.05 172810.0
172850.0 172721.05 172860.0
172897.0 172771.05 172907.0
172948.0 172820.94 172958.0
172999.0 172868.06 173009.0
173052.0 172919.05 173062.0
173103.0 172970.1 173113.0
173152.0 173023.05 173162.0
173199.0 173073.95 173209.0
173248.0 173122.9 173258.0
173298.0 173169.94 173308.0
173350.0 173218.95 173360.0
173399.0 173269.05 173409.0
173450.0 173320.94 173460.0
173502.0 173369.98 173512.0
173553.0 173421.05 173563.0
173602.0 173472.95 173612.0
173653.0 173523.9 173663.0
173701.0 173572.95 173711.0


169994.0 169964.78 170004.0
169995.0 169964.77 170005.0
169994.0 169963.81 170004.0
169995.0 169964.77 170005.0
169994.0 169963.81 170004.0
169994.0 169964.77 170004.0
169994.0 169963.78 170004.0
169994.0 169963.78 170004.0
169995.0 169963.78 170005.0
169995.0 169963.81 170005.0
169995.0 169964.78 170005.0
169995.0 169964.78 170005.0
169994.0 169964.78 170004.0
169994.0 169964.77 170004.0
169994.0 169963.78 170004.0
169994.0 169963.78 170004.0
169994.0 169963.78 170004.0
169994.0 169963.78 170004.0
169995.0 169963.78 170005.0
169995.0 169963.81 170005.0
169995.0 169964.78 170005.0
169995.0 169964.78 170005.0
169994.0 169964.78 170004.0
169994.0 169964.77 170004.0
169994.0 169963.78 170004.0
169994.0 169963.78 170004.0
169994.0 169963.78 170004.0
169994.0 169963.78 170004.0
169994.0 169963.78 170004.0
169994.0 169963.78 170004.0
169995.0 169963.78 170005.0
169995.0 169963.81 170005.0
169995.0 169964.78 170005.0
169994.0 169964.78 170004.0
169994.0 169964.77 170004.0
169994.0 169963.78 1

-13.0 -2.78 -3.0
-13.0 -2.69 -3.0
-14.0 -2.61 -4.0
-14.0 -2.61 -4.0
-14.0 -2.7 -4.0
-14.0 -2.7 -4.0
-13.0 -2.7 -3.0
-14.0 -2.69 -4.0
-13.0 -2.61 -3.0
-13.0 -2.69 -3.0
-13.0 -2.61 -3.0
-13.0 -2.61 -3.0
-13.0 -2.61 -3.0
-13.0 -2.61 -3.0
-13.0 -2.61 -3.0
-13.0 -2.61 -3.0
-13.0 -2.61 -3.0
-13.0 -2.61 -3.0
-11.0 -2.61 -1.0
-7.0 -2.6 3.0
-5.0 -2.42 5.0
-6.0 -2.04 4.0
-7.0 -1.85 3.0
-9.0 -1.95 1.0
-8.0 -2.05 2.0
-7.0 -2.23 3.0
-6.0 -2.14 4.0
-6.0 -2.04 4.0
-7.0 -1.95 3.0
-8.0 -1.95 2.0
-8.0 -2.05 2.0
-7.0 -2.14 3.0
-6.0 -2.14 4.0
-6.0 -2.04 4.0
-7.0 -1.95 3.0
-7.0 -1.95 3.0
-8.0 -2.04 2.0
-7.0 -2.05 3.0
-7.0 -2.14 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -2.04 3.0
-7.0 -

164895.0 164767.0 164905.0
164945.0 164817.0 164955.0
164994.0 164867.0 165004.0
165043.0 164916.94 165053.0
165093.0 164965.94 165103.0
165144.0 165014.97 165154.0
165193.0 165064.98 165203.0
165244.0 165115.94 165254.0
165293.0 165165.0 165303.0
165344.0 165215.9 165354.0
165394.0 165264.97 165404.0
165445.0 165315.92 165455.0
165494.0 165365.97 165504.0
165544.0 165416.88 165554.0
165594.0 165465.9 165604.0
165645.0 165515.9 165655.0
165694.0 165565.94 165704.0
165744.0 165616.86 165754.0
165794.0 165665.89 165804.0
165844.0 165715.89 165854.0
165895.0 165765.86 165905.0
165944.0 165815.9 165954.0
165993.0 165866.83 166003.0
166042.0 165915.83 166052.0
166092.0 165964.81 166102.0
166143.0 166013.84 166153.0
166193.0 166063.88 166203.0
166243.0 166114.84 166253.0
166292.0 166164.81 166302.0
166343.0 166214.8 166353.0
166393.0 166263.84 166403.0
166444.0 166314.81 166454.0
166493.0 166364.83 166503.0
166543.0 166415.78 166553.0
166593.0 166464.78 166603.0
166644.0 166514.78 166654.0
1

56239.0 56321.9 56249.0
56188.0 56272.81 56198.0
56139.0 56220.85 56149.0
56090.0 56169.91 56100.0
56041.0 56120.92 56051.0
55990.0 56071.93 56000.0
55940.0 56022.87 55950.0
55890.0 55971.91 55900.0
55841.0 55921.92 55851.0
55791.0 55871.96 55801.0
55740.0 55822.92 55750.0
55689.0 55772.9 55699.0
55640.0 55721.9 55650.0
55591.0 55670.98 55601.0
55542.0 55621.98 55552.0
55491.0 55572.98 55501.0
55440.0 55523.93 55450.0
55390.0 55472.94 55400.0
55340.0 55421.98 55350.0
55291.0 55371.98 55301.0
55240.0 55322.02 55250.0
55188.0 55272.96 55198.0
55139.0 55221.94 55149.0
55090.0 55170.03 55100.0
55041.0 55121.04 55051.0
54990.0 55072.04 55000.0
54939.0 55022.99 54949.0
54890.0 54972.0 54900.0
54840.0 54921.06 54850.0
54791.0 54872.04 54801.0
54739.0 54822.08 54749.0
54688.0 54772.99 54698.0
54640.0 54721.02 54650.0
54590.0 54670.13 54600.0
54541.0 54622.07 54551.0
54489.0 54572.11 54499.0
54439.0 54523.02 54449.0
54389.0 54471.08 54399.0
54340.0 54421.1 54350.0
54289.0 54371.13 54299.0
54237

74938.0 75021.56 74948.0
74888.0 74969.56 74898.0
74839.0 74917.64 74849.0
74790.0 74867.66 74800.0
74740.0 74818.68 74750.0
74690.0 74769.65 74700.0
74640.0 74719.65 74650.0
74589.0 74669.66 74599.0
74539.0 74619.64 74549.0
74489.0 74568.68 74499.0
74439.0 74518.68 74449.0
74389.0 74468.68 74399.0
74340.0 74418.7 74350.0
74289.0 74368.72 74299.0
74237.0 74319.67 74247.0
74187.0 74268.65 74197.0
74138.0 74216.71 74148.0
74091.0 74166.76 74101.0
74042.0 74117.82 74052.0
73989.0 74070.76 73999.0
73938.0 74021.64 73948.0
73889.0 73968.71 73899.0
73839.0 73917.78 73849.0
73791.0 73868.76 73801.0
73740.0 73818.82 73750.0
73689.0 73770.74 73699.0
73639.0 73719.75 73649.0
73589.0 73668.78 73599.0
73539.0 73618.79 73549.0
73489.0 73568.8 73499.0
73439.0 73518.8 73449.0
73388.0 73468.8 73398.0
73339.0 73418.78 73349.0
73289.0 73367.85 73299.0
73237.0 73318.82 73247.0
73186.0 73268.76 73196.0
73138.0 73216.8 73148.0
73090.0 73165.9 73100.0
73040.0 73117.92 73050.0
72988.0 73069.86 72998.0
72937.

72790.0 72866.94 72800.0
72739.0 72818.92 72749.0
72688.0 72769.86 72698.0
72638.0 72718.86 72648.0
72588.0 72667.9 72598.0
72539.0 72617.9 72549.0
72488.0 72567.94 72498.0
72438.0 72518.89 72448.0
72389.0 72467.93 72399.0
72339.0 72417.96 72349.0
72288.0 72368.93 72298.0
72237.0 72318.92 72247.0
72186.0 72267.92 72196.0
72138.0 72216.92 72148.0
72090.0 72166.02 72100.0
72040.0 72118.03 72050.0
71989.0 72069.98 71999.0
71937.0 72019.94 71947.0
71887.0 71968.93 71897.0
71839.0 71916.98 71849.0
71790.0 71867.06 71800.0
71740.0 71819.04 71750.0
71689.0 71770.01 71699.0
71639.0 71719.98 71649.0
71589.0 71669.02 71599.0
71539.0 71619.02 71549.0
71489.0 71569.03 71499.0
71439.0 71519.04 71449.0
71389.0 71469.04 71399.0
71340.0 71419.06 71350.0
71289.0 71369.1 71299.0
71238.0 71320.03 71248.0
71187.0 71269.04 71197.0
71138.0 71218.06 71148.0
71090.0 71167.11 71100.0
71041.0 71118.15 71051.0
70989.0 71070.12 70999.0
70938.0 71021.03 70948.0
70888.0 70969.07 70898.0
70839.0 70918.11 70849.0
707

KeyboardInterrupt: 

In [190]:
for i in range(len(Y_ind)):
    print(min_koord['x osa'][Y_ind][2+2*i], train_predict[i], max_koord['x osa'][Y_ind][2+2*i])

-1 [-1.8475676] 1
2 [-1.8475676] 12
14 [-1.8404282] 31
25 [-1.6416923] 37
29 [-1.3418055] 39
40 [-0.84318614] 54
54 [-0.05538796] 74
69 [3.4281206] 108
91 [11.7864275] 116
115 [14.032848] 135
133 [14.067701] 158
181 [15.2606125] 207
227 [17.716148] 244
241 [25.117577] 256
257 [47.294983] 280
307 [74.80958] 333
375 [92.99] 394
414 [99.833824] 430
426 [103.23827] 451
472 [104.57061] 492
535 [111.79778] 553
602 [128.27104] 618
647 [149.36505] 662
685 [173.9227] 702
737 [203.59402] 752
801 [231.69652] 819
863 [242.24126] 883
923 [250.60085] 945
986 [254.93935] 996
1044 [262.68265] 1056
1102 [280.97064] 1125
1177 [307.58646] 1198
1258 [337.32123] 1268
1322 [368.18036] 1335
1385 [400.53915] 1406
1462 [422.17963] 1479
1550 [435.07245] 1573
1639 [446.0038] 1657
1713 [456.16074] 1724
1788 [472.32928] 1798
1876 [495.50607] 1899
1969 [526.46545] 1992
2065 [559.1642] 2090
2148 [585.7948] 2172
2241 [613.935] 2261
2335 [645.05145] 2355
2438 [664.9873] 2460
2545 [684.92316] 2569
2641 [704.9543] 2658


9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985

9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9984.993] 10003
9995 [9984.023] 10003
9995 [9984.056] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985

9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985

9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985.024] 10003
9995 [9985

...

122745 [48181.02] 122750
122643 [48230.984] 122650
122543 [48280.04] 122551
122444 [48331.004] 122450
122344 [48380.996] 122350
122243 [48430.965] 122251
122142 [48479.953] 122150
122044 [48528.98] 122051
121944 [48579.004] 121951
121845 [48629.965] 121853
121746 [48679.96] 121752
121645 [48729.953] 121652
121546 [48779.98] 121553
121445 [48830.94] 121452
121346 [48880.902] 121353
121246 [48929.902] 121253
121145 [48978.92] 121152
121046 [49028.92] 121053
120947 [49078.914] 120953
120847 [49128.94] 120854
120748 [49179.867] 120754
120646 [49228.89] 120653
120541 [49278.89] 120554
120435 [49328.914] 120453
120345 [49379.875] 120353
120245 [49429.87] 120254
120136 [49479.863] 120152
120040 [49529.83] 120053
119944 [49578.914] 119953
119842 [49630.844] 119855
119742 [49680.844] 119756
119642 [49730.805] 119654
119541 [49779.83] 119555
119434 [49829.85] 119454
119341 [49880.812] 119355
119244 [49930.777] 119254
119136 [49979.77] 119153
119040 [50028.766] 119054
118943 [50077.82] 118954
118

89542 [64780.023] 89553
89440 [64830.05] 89451
89340 [64880.98] 89351
89242 [64929.973] 89250
89139 [64978.938] 89151
89041 [65026.996] 89051
88939 [65077.02] 88951
88840 [65128.016] 88852
88742 [65178.945] 88751
88639 [65227.97] 88651
88542 [65277.996] 88552
88439 [65328.957] 88451
88339 [65378.953] 88351
88241 [65428.945] 88249
88138 [65478.945] 88149
88042 [65528.938] 88051
87937 [65578.93] 87949
87840 [65628.914] 87852
87741 [65678.945] 87751
87639 [65729.875] 87651
87541 [65778.9] 87552
87439 [65828.93] 87450
87339 [65879.86] 87351
87241 [65928.85] 87250
87139 [65977.85] 87149
87041 [66026.836] 87051
86939 [66075.9] 86950
86839 [66126.89] 86851
86741 [66177.82] 86750
86639 [66226.82] 86649
86541 [66275.875] 86551
86438 [66326.87] 86449
86338 [66377.87] 86348
86240 [66428.8] 86247
86138 [66477.79] 86147
86043 [66526.79] 86051
85940 [66575.87] 85947
85839 [66627.836] 85849
85744 [66678.76] 85750
85642 [66727.76] 85649
85542 [66776.78] 85549
85441 [66826.805] 85449
85342 [66877.77] 8

51643 [83732.77] 51650
51544 [83783.77] 51551
51442 [83834.7] 51450
51343 [83883.664] 51351
51242 [83931.82] 51249
51142 [83984.75] 51149
51043 [84035.766] 51051
50941 [84087.695] 50950
50843 [84137.63] 50851
50745 [84185.6] 50750
50643 [84232.625] 50651
50544 [84280.77] 50551
50443 [84333.77] 50450
50344 [84386.64] 50351
50243 [84435.625] 50250
50143 [84484.69] 50150
50044 [84535.74] 50051
49943 [84588.65] 49951
49844 [84638.58] 49851
49745 [84686.54] 49751
49644 [84733.63] 49652
49545 [84783.68] 49552
49444 [84835.65] 49451
49344 [84886.54] 49351
49244 [84934.6] 49250
49143 [84984.66] 49150
49044 [85036.69] 49051
48944 [85089.46] 48950
48844 [85135.54] 48851
48745 [85184.63] 48751
48644 [85236.5] 48651
48545 [85284.56] 48552
48443 [85334.46] 48450
48343 [85381.35] 48351
48243 [85425.54] 48250
48142 [85475.695] 48150
48043 [85530.73] 48051
47943 [85586.46] 47950
47843 [85634.52] 47851
47744 [85684.414] 47750
47642 [85731.38] 47651
47543 [85777.52] 47551
47442 [85828.555] 47449
47342 [

15240 [99960.43] 15247
15138 [99967.17] 15146
15039 [99966.15] 15048
14938 [99964.24] 14946
14839 [99965.305] 14847
14740 [99968.67] 14747
14639 [99983.62] 14647
14540 [99996.3] 14547
14437 [99998.914] 14447
14338 [99989.85] 14348
14240 [99979.05] 14247
14138 [99974.26] 14147
14040 [99976.336] 14050
13938 [99980.336] 13947
13840 [99984.336] 13848
13740 [99988.18] 13749
13639 [99987.07] 13649
13542 [99983.08] 13548
13438 [99979.234] 13449
13339 [99980.305] 13349
13241 [99983.24] 13247
13139 [99984.21] 13148
13041 [99984.21] 13050
12939 [99984.18] 12948
12840 [99983.14] 12849
12742 [99981.195] 12749
12640 [99981.26] 12649
12540 [99983.21] 12549
12439 [99982.95] 12448
12340 [99974.85] 12348
12240 [99964.055] 12247
12139 [99959.305] 12147
12040 [99962.36] 12049
11938 [99967.27] 11947
11839 [99969.195] 11848
11741 [99969.164] 11748
11640 [99968.13] 11648
11542 [99966.15] 11548
11439 [99964.27] 11448
11340 [99966.27] 11349
11241 [99968.195] 11248
11140 [99968.164] 11148
11041 [99967.195] 110

-6 [85962.27] 1
-6 [85911.336] 1
-6 [85862.38] 1
-6 [85814.35] 1
-6 [85765.266] 1
-6 [85713.336] 1
-6 [85663.305] 1
-6 [85612.375] 1
-6 [85563.414] 1
-6 [85515.36] 1
-6 [85465.3] 1
-6 [85413.37] 1
-6 [85363.34] 1
-6 [85312.445] 1
-6 [85264.39] 1
-6 [85214.4] 1
-6 [85164.37] 1
-6 [85113.41] 1
-6 [85063.48] 1
-6 [85015.42] 1
-6 [84965.39] 1
-6 [84914.43] 1
-6 [84864.4] 1
-6 [84813.5] 1
-6 [84765.48] 1
-6 [84716.484] 1
-6 [84667.45] 1
-6 [84617.46] 1
-6 [84567.48] 1
-6 [84517.445] 1
-6 [84466.48] 1
-6 [84416.49] 1
-6 [84366.46] 1
-6 [84315.54] 1
-6 [84266.51] 1
-6 [84216.51] 1
-6 [84166.52] 1
-6 [84116.586] 1
-6 [84068.6] 1
-6 [84020.445] 1
-6 [83967.45] 1
-6 [83914.586] 1
-6 [83865.62] 1
-6 [83817.586] 1
-6 [83768.52] 1
-6 [83717.54] 1
-6 [83666.58] 1
-6 [83616.62] 1
-6 [83567.586] 1
-6 [83517.53] 1
-6 [83465.57] 1
-6 [83414.61] 1
-6 [83364.58] 1
-6 [83313.65] 1
-6 [83264.625] 1
-6 [83214.63] 1
-6 [83164.63] 1
-6 [83114.67] 1
-6 [83065.68] 1
-6 [83016.6] 1
-6 [82964.6] 1
-6 [82912.664] 1